# OpenAI Function Calling


Let's learn how to connect LLMs to external tools
* In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. 
* Function calling does NOT mean to call function, but the LLM output a JSON object conatining arguments to call one or many functions based on the input text. 
* All you have to do is describing the function and its arguments for the LLM and it will detect depending on the input when the function should be called and repond with JSON.

**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [5]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [7]:
print(response)

{
  "id": "chatcmpl-8IGfQeACLdZTwquVmjsVT1x9aOtIS",
  "object": "chat.completion",
  "created": 1699364664,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}


In [8]:
response_message = response["choices"][0]["message"]

In [9]:
response_message

<OpenAIObject at 0x2268500eac0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}

In [10]:
response_message["content"]

In [11]:
response_message["function_call"]

<OpenAIObject at 0x2268500f6a0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
}

In [12]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

> Function calling does NOT mean to call function, but the LLM output a JSON object conatining arguments to call one or many functions based on the input text. 

In [13]:
args = json.loads(response_message["function_call"]["arguments"])

In [14]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [15]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [17]:
print(response)

{
  "id": "chatcmpl-8IGgKJ6PeiZRstzr0A05fmFrzMPLq",
  "object": "chat.completion",
  "created": 1699364720,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


* The default behavior (``tool_choice: "auto"``) is for the model to decide on its own whether to call a function and if so which function to call.
* If you want to force the model to call a specific function you can do so by setting ``tool_choice`` with a specific function name. You can also force the model to generate a user-facing message by setting ``tool_choice: "none"``

In [18]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-8IGjwVw86cinFiZZ16t6QU0VhBDyT",
  "object": "chat.completion",
  "created": 1699364944,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8IGkCrSPXZkw1T4YQGPNzuYkRvtdn",
  "object": "chat.completion",
  "created": 1699364960,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  }
}


In [20]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8IGkQEJZEzrXYfVA0dRekggUwQAUS",
  "object": "chat.completion",
  "created": 1699364974,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"location\": \"Boston, MA\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 11,
    "total_tokens": 93
  }
}


* Force the model to call a specific function. Even though there is no reason to detect function, but the model will return with JSON from function description we provided before:

In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613", 
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8IGmSztSaaFwThNh7N86Fc3fKaSWt",
  "object": "chat.completion",
  "created": 1699365100,
  "model": "gpt-4-1106-preview",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"location\":\"San Francisco, CA\",\"unit\":\"fahrenheit\"}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 86,
    "completion_tokens": 13,
    "total_tokens": 99
  },
  "system_fingerprint": "fp_a24b4d720c"
}


In [23]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8IGmzJqijzQNhpKY0HImSbrkdRBkP",
  "object": "chat.completion",
  "created": 1699365133,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  }
}


In [26]:
# Append the response to the messages list 
# [{'role': 'user', 'content': "What's the weather like in Boston!"}]
messages.append(response["choices"][0]["message"])

In [27]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 <OpenAIObject at 0x2268503fc90> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
   }
 }]

In [28]:
# Simulate calling the funtcion with the arguments from the LLM
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

Let's append a new message with ``"role":"function"``, `"name"` of a specific function, `"content"` representing the function output

In [29]:
# Append new message representing the funtion output

messages.append(
        {
            "role": "function", # new role	
            "name": "get_current_weather",
            "content": observation,
        }
)

In [30]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 <OpenAIObject at 0x2268503fc90> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
   }
 },
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

If we then call the LLM with the list of messages, we can see the LLM takes the reponse of the function and convert it to natural language response:

In [31]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-8IH8BKSlUlq4KTFBqEthCg7SWW81v",
  "object": "chat.completion",
  "created": 1699366447,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72\u00b0F."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 17,
    "total_tokens": 94
  }
}
